<a href="https://colab.research.google.com/github/Kalash8286/Scraping/blob/main/web_scraping/WebScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import files  
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString
import csv  
import pandas as pd

In [ ]:
#variables
subHeading, subHeadingList  =([] for i in range(2))
intro = ''
chapterName = ''

#final answer list
subHeadingIntro, subHeadings, allEaxmples =([] for i in range(3))

In [ ]:
#convert list into string
def listIntoString(lists):    
    str1 =''
    for ele in lists:         
        str1 += ele  
    return str1

In [ ]:
# here all p tags are passed which includes text all other headings also
# so here exatracting only needed text for 
def filterSubHeadingIntro(subHeadingIntro):
    removeStr = len(subHeadingIntro)
    i = -1
    subIntro = []
    lastItem = ""
    while removeStr > 0:   
        if( len(subHeadingIntro[i]) != 0 ):
            if(len(subIntro) == 0):
                subIntro.append(subHeadingIntro[i])
                lastItem = subHeadingIntro[i]
            else:
                subIntro.append(subHeadingIntro[i].replace(lastItem, ""))
                lastItem = subHeadingIntro[i]           
        i -= 1
        removeStr -= 1
    subIntro.reverse()
    return subIntro

In [ ]:
def findHeading(soup):    
  # print("\n==========PAGE HEADING==============")
  heading = soup.find_all('span', class_='color_h1')
  if(heading != []):
    heading = heading[0].get_text()
  else:
    heading = ""  
  

  # print("\n==========INTRODUCTION==============")
  global intro 
  intro = soup.find_all('p', class_='intro')
  
  if(intro != []):
    intro = intro[0].get_text()
  else:
    intro = ""  
  
  return heading

In [ ]:
def findSubHeading(soup):
  global subHeadings
  try:
    # print("\n==========ALL SUB HEADINGS==============")
    sub_heading = soup.find('hr')
    sub_heading = sub_heading.find_next_siblings("h2")
    for h in sub_heading:
        if (h.string.find('Exercises') != -1):
            break    
        elif(h.string == 'Chapter Summary'):
            break
        elif(h.string == 'HTML List Tags'):
            break
        else:
            subHeadings.append(h.get_text())
  except:
    subHeadings.append('')

In [ ]:
def findIntroSubheading(soup):
  global subHeadingIntro
  # print("\n==========INTRO FOR ALL SUB HEADINGS==============")
  try:
    sub_heading_intro = soup.find('div', id='main')
    length = len(sub_heading_intro.find_all('hr'))
    i=0
    while i < length:
        sub = sub_heading_intro.find_all('hr')[i]
        sub = sub.find_next_siblings("p")
        for k in sub:      
            if(k.get_text() == intro):
                continue
            if(k.get_text().__contains__("W3Schools' tag reference contains")):
                continue
            else:  
                subHeadingList.append(k.get_text()) 
        str1 = listIntoString(subHeadingList)
        subHeadingList.clear()  
        if(str1 not in subHeadingIntro):
            subHeadingIntro.append(str1)               
        i += 1
  except:
    subHeadingIntro.append('')               
    # sending data for filtering
    subHeadingIntro = filterSubHeadingIntro(subHeadingIntro)

In [ ]:
def findExample(soup):
  global allEaxmples
  dummyExamples = []
  text = ''

  # print("\n==========ALL EXAMPLES FOR SUB HEADINGS==============")
  example = soup.select("div.w3-example div.notranslate")
  for e in example:    
    example = False            
    dummyExamples.append(e.get_text())        
    par = e.parent     
  
    #under one heading can have multiple examples
    for sibling in par.next_siblings: 
      # print(sibling)
      for s in sibling:
        if(str(s) == '<h3>Example</h3>'):
          example = True
      if(example == False):
        if(str(sibling) == '<hr/>'):
          dummyExamples.append("|||") 
          break

  # arranging all examples  
  for e in dummyExamples:    
    if(e == '|||'):
      allEaxmples.append(text)
      text = ''
    else:
      if(len(text) == 0):
        text += e
      else:
        text += ' &&&&&& '+e 

In [ ]:
def writeInCSV(chapterName, heading, url, subHeadings, subHeadingIntro, allEaxmples):   

  
  for k in range(len(subHeadings)):
    if(k >= len(subHeadingIntro)):
      subHeadingIntro.append("")
    if(k >= len(allEaxmples)):
      allEaxmples.append("")
 

  with open('web.csv', 'a', encoding='UTF8') as f:
    writer = csv.writer(f)
    counter = 0 
    heading = chapterName +'||'+ heading.strip()     
    # print(heading)
    for head in subHeadings:
      data = heading, url, subHeadings[counter], subHeadingIntro[counter], allEaxmples[counter]
      writer.writerow(data)
      counter += 1

In [ ]:
header = ['Heading', 'link', 'Subheading', 'Definations', 'Example']
with open('web.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header)  



In [ ]:
webLinks = pd.read_csv('link.txt',sep='\n',header=None)[0].tolist()

num = 1

for link in webLinks:  
  chapterName = link.split("/")[3]
  r = requests.get(link)   
  soup = BeautifulSoup(r.content, 'html.parser')
  heading = findHeading(soup)
  findSubHeading(soup)  
  findIntroSubheading(soup)
  findExample(soup)
  writeInCSV(chapterName, heading, link, subHeadings, subHeadingIntro, allEaxmples)  
  subHeadingIntro.clear()
  subHeadings.clear()
  allEaxmples.clear()  
  print(num, 'RECORD DONE!')
  num += 1


In [ ]:
# df = pd.read_csv("web.csv")  
# df.head(10)

In [ ]:
# file1 = open("link.txt","a")

# URL = "https://www.w3schools.com/cpp/"
# r = requests.get(URL)   
# soup = BeautifulSoup(r.content, 'html.parser')

# divs  = soup.find("div", id="leftmenuinnerinner")
# # soup.select("div.w3-example div.notranslate")

# for a in divs.find_all('a', href=True, target="_top"):
#   links = URL + a['href']
#   file1.write(links)
#   file1.write('\n')
#   print(links)

# file1.close()